In [0]:
import numpy as np
from keras.preprocessing import image

Using TensorFlow backend.


In [0]:

import os, shutil
folders = os.listdir("/content/drive/My Drive/Images")
print(folders)


['pikachu', 'Charmander', 'bulbasaur']


In [0]:
image_data = []
labels = []
label_dict = {
    "pikachu":0,
    "bulbasaur":1,
    "Charmander":2,
    
}

In [0]:
for ix in folders:
  path = os.path.join("/content/drive/My Drive/Images",ix)
  for im in os.listdir(path):
    img = image.load_img(os.path.join(path,im),target_size = ((224,224)))
    img_array = image.img_to_array(img)
    image_data.append(img_array)
    labels.append(label_dict[ix]) 

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [0]:
print(len(image_data),len(labels))

1187 1187


In [0]:
import random
combined = list(zip(image_data,labels))
random.shuffle(combined)
image_data[:],labels[:] = zip(*combined)

In [0]:
print(labels[:5])

[2, 2, 2, 2, 0]


In [0]:
x_train = np.array(image_data)
y_train = np.array(labels)


print(x_train.shape,y_train.shape)

(1187, 224, 224, 3) (1187,)


In [0]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
print(x_train.shape,y_train.shape)

(1187, 224, 224, 3) (1187, 3)


In [0]:
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Model
from matplotlib import pyplot as plt

In [0]:
model = ResNet50(include_top = False,weights = 'imagenet',input_shape = (224,224,3))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [0]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [0]:
av1 = GlobalAveragePooling2D()(model.output) ##keras functional api
fc1 = Dense(256,activation='relu')(av1)
d1 = Dropout(0.6)(fc1)
fc2 = Dense(3,activation='softmax')(d1)

model_new = Model(input=model.input,outputs = fc2)
model_new.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  


In [0]:
adam = Adam(lr=0.003)
model_new.compile(loss = 'categorical_crossentropy',optimizer=adam,metrics = ['accuracy'])

In [0]:
for ix in range(len(model_new.layers)):
  print(ix,model_new.layers[ix])

0 <keras.engine.input_layer.InputLayer object at 0x7f80f0badcc0>
1 <keras.layers.convolutional.ZeroPadding2D object at 0x7f80f0bad5f8>
2 <keras.layers.convolutional.Conv2D object at 0x7f80f0badef0>
3 <keras.layers.normalization.BatchNormalization object at 0x7f80f0bbcbe0>
4 <keras.layers.core.Activation object at 0x7f80f0bd67f0>
5 <keras.layers.convolutional.ZeroPadding2D object at 0x7f80f0bd1be0>
6 <keras.layers.pooling.MaxPooling2D object at 0x7f80f0b8bf98>
7 <keras.layers.convolutional.Conv2D object at 0x7f80f0b39128>
8 <keras.layers.normalization.BatchNormalization object at 0x7f80f0b394a8>
9 <keras.layers.core.Activation object at 0x7f80f0b39e80>
10 <keras.layers.convolutional.Conv2D object at 0x7f80f0b3f6a0>
11 <keras.layers.normalization.BatchNormalization object at 0x7f80f0af5588>
12 <keras.layers.core.Activation object at 0x7f80f0afffd0>
13 <keras.layers.convolutional.Conv2D object at 0x7f80f0b046a0>
14 <keras.layers.convolutional.Conv2D object at 0x7f80f0ac4f28>
15 <keras.lay

In [0]:
 for ix in range(155):
   model_new.layers[ix].trainable = False

In [0]:
model_new.compile(loss = 'categorical_crossentropy',optimizer=adam,metrics = ['accuracy'])

In [0]:
model_new.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [0]:
hist = model_new.fit(x_train,y_train,
                     shuffle = True,
                     batch_size = 100,
                     epochs = 15,
                     validation_split = 0.10)

Train on 1068 samples, validate on 119 samples
Epoch 1/15
1068/1068 [==============================] - 19s 17ms/step - loss: 0.8016 - acc: 0.8071 - val_loss: 4.9188 - val_acc: 0.6891
Epoch 2/15
1068/1068 [==============================] - 5s 5ms/step - loss: 0.0851 - acc: 0.9710 - val_loss: 0.7333 - val_acc: 0.9496
Epoch 3/15
1068/1068 [==============================] - 5s 5ms/step - loss: 0.0350 - acc: 0.9860 - val_loss: 0.4702 - val_acc: 0.9496
Epoch 4/15
1068/1068 [==============================] - 5s 5ms/step - loss: 0.0255 - acc: 0.9925 - val_loss: 0.7317 - val_acc: 0.9244
Epoch 5/15
1068/1068 [==============================] - 5s 5ms/step - loss: 0.0148 - acc: 0.9944 - val_loss: 0.1150 - val_acc: 0.9748
Epoch 6/15
1068/1068 [==============================] - 5s 5ms/step - loss: 0.0089 - acc: 0.9972 - val_loss: 0.0576 - val_acc: 0.9832
Epoch 7/15
1068/1068 [==============================] - 5s 5ms/step - loss: 0.0085 - acc: 0.9972 - val_loss: 0.0698 - val_acc: 0.9748
Epoch 8/15
10

In [0]:
test_image = []
test_labels = []

In [0]:


path = os.path.join("/content/drive/My Drive/new_test","")
for im in os.listdir(path):
  img = image.load_img(os.path.join(path,im),target_size = ((224,224)))
  img_array = image.img_to_array(img)
  test_image.append(img_array)
  test_labels.append(im) 
print(len(test_image),len(test_labels))    

123 123


In [0]:
test_image = np.array(test_image)
out = model_new.predict(test_image)

In [0]:
print(out)

[[3.17947212e-07 9.99986172e-01 1.34729817e-05]
 [6.86229029e-09 9.99997616e-01 2.43090130e-06]
 [1.09365472e-09 9.99998808e-01 1.20021230e-06]
 [5.94428172e-07 9.99922514e-01 7.69261896e-05]
 [4.56583109e-08 9.99998450e-01 1.53311930e-06]
 [1.63945093e-16 1.00000000e+00 8.73475446e-13]
 [1.03074244e-05 9.99851108e-01 1.38574556e-04]
 [2.03783060e-07 9.99994397e-01 5.36865355e-06]
 [1.79456487e-15 1.00000000e+00 8.99380559e-13]
 [1.92099620e-07 9.99634027e-01 3.65777494e-04]
 [4.56583109e-08 9.99998450e-01 1.53311930e-06]
 [2.42156251e-09 9.99997258e-01 2.77798176e-06]
 [9.56975384e-12 1.00000000e+00 3.11674886e-09]
 [4.48831305e-10 1.00000000e+00 2.00849861e-08]
 [1.24156848e-02 8.33461463e-01 1.54122904e-01]
 [1.54747784e-06 9.99884725e-01 1.13667680e-04]
 [4.65306766e-06 9.99928355e-01 6.70195441e-05]
 [7.65235011e-12 1.00000000e+00 6.46535669e-09]
 [2.86573595e-07 9.99972463e-01 2.73287551e-05]
 [9.99999523e-01 1.38696990e-07 4.11517817e-07]
 [3.56785998e-13 1.00000000e+00 2.832704

In [0]:
test_labels

['1102.jpg',
 '1108.jpg',
 '1107.jpg',
 '1105.jpg',
 '1100.jpg',
 '1101.jpg',
 '1106.jpg',
 '1104.jpg',
 '1103.jpg',
 '1116.jpg',
 '1113.jpg',
 '1112.jpg',
 '1110.jpg',
 '1115.jpg',
 '1117.jpg',
 '1118.jpg',
 '1119.jpg',
 '1114.jpg',
 '1111.jpg',
 '2019.jpg',
 '1124.jpg',
 '1127.jpg',
 '1123.jpg',
 '1109.jpg',
 '1126.jpg',
 '1128.jpg',
 '1120.jpg',
 '1129.jpg',
 '2203.jpg',
 '1134.jpg',
 '1136.jpg',
 '1131.jpg',
 '2214.jpg',
 '1130.jpg',
 '1125.jpg',
 '1121.jpg',
 '1132.jpg',
 '1139.jpg',
 '1133.jpg',
 '1135.jpg',
 '1138.jpg',
 '1137.jpg',
 '1122.jpg',
 '2002.jpg',
 '2005.jpg',
 '2000.jpg',
 '2004.jpg',
 '2006.jpg',
 '2003.jpg',
 '2001.jpg',
 '2008.jpg',
 '2007.jpg',
 '1140.jpg',
 '2034.jpg',
 '2039.jpg',
 '2036.jpg',
 '2038.jpg',
 '2035.jpg',
 '2032.jpg',
 '2033.jpg',
 '2030.jpg',
 '2024.jpg',
 '2027.jpg',
 '2029.jpg',
 '2021.jpg',
 '2020.jpg',
 '2025.jpg',
 '2028.jpg',
 '2026.jpg',
 '2023.jpg',
 '2022.jpg',
 '2010.jpg',
 '2014.jpg',
 '2009.jpg',
 '2017.jpg',
 '2011.jpg',
 '2018.jpg',

In [0]:
print(test_labels[100],out[100])

2223.jpg [3.2684630e-07 7.1433415e-06 9.9999249e-01]


In [0]:
import torch
model_save_name = 'pokemonGoa.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model_new.state_dict(), path)

AttributeError: ignored

In [0]:
import pandas as pd
df = pd.DataFrame(out)
col = df.idxmax(axis=1)
output = []
for colm in col:
  if(colm==0):
    output.append("Pikachu")
  if(colm==1):
    output.append("Bulbasaur")
  if(colm==2):
    output.append("Charmander")

In [0]:
output

['Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Pikachu',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Charmander',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Charmander',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Bulbasaur',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Charmander',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu

In [0]:
df = pd.DataFrame(list(zip(test_labels, output)), 
               columns =['ImageId','NameOfPokemon'])

In [0]:
df

,ImageId,NameOfPokemon
0,1102.jpg,Bulbasaur
1,1108.jpg,Bulbasaur
2,1107.jpg,Bulbasaur
3,1105.jpg,Bulbasaur
4,1100.jpg,Bulbasaur
...,...,...
118,2232.jpg,Charmander
119,2234.jpg,Charmander
120,2231.jpg,Charmander
121,2224.jpg,Charmander


In [0]:
df.to_csv('prediction.csv',index =False)


In [0]:
test = pd.read_csv("/content/test.csv").values
pred = pd.read_csv("/content/prediction.csv").values

In [0]:
new_prediction = []
for id in test:
  for i in range(124):
    if id == pred[i][0]:
      new_prediction.append(pred[i][1])
      break


In [0]:
new_prediction

['Bulbasaur',
 'Charmander',
 'Bulbasaur',
 'Charmander',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Bulbasaur',
 'Bulbasaur',
 'Charmander',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Charmander',
 'Pikachu',
 'Charmander',
 'Charmander',
 'Pikachu',
 'Charmander',
 'Pikachu',
 'Bulbasaur',
 'Charmander',
 'Bulbasaur',
 'Charmander',
 'Charmander',
 'Bulbasaur',
 'Bulbasaur',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Pikachu',
 'Charmander',
 'Bulbasaur',
 'Pikachu',
 'Bulbasaur',
 'Bulbasaur',
 'Bulbasaur',
 'Charmander',
 'Bulbasaur',
 'Bulbasaur',
 'Charmander',
 'Pikachu',
 'Charmander',
 'Charmander',
 'Bulbasaur',
 'Charmander',
 'Bulbasaur',
 'Charmander',
 'Charmander',
 'Pikachu',
 'Charmander',
 'Pikachu',
 'Bulbasaur',
 'Charmander',
 'Bulbasaur',
 'Charmander',
 'Pikachu',
 'Pikachu',
 'Charmander',
 'Charmander',
 'Pikachu',
 'Charmander',
 'Charmander',
 'Pikachu',
 'Pikachu',
 'Bulbasaur',
 'Pikachu',
 'Pikachu',
 'Bulbasaur',
 'Charmander',
 'Bulbasaur',
 'Bulbas

In [0]:
df2 = pd.DataFrame(test,columns =['ImageId'])

In [0]:
df2

,ImageId
0,1106.jpg
1,2236.jpg
2,1131.jpg
3,2221.jpg
4,2011.jpg
...,...
118,1124.jpg
119,2014.jpg
120,2235.jpg
121,1126.jpg


In [0]:
df2["NameOfPokemon"] = new_prediction

In [0]:
df2

,ImageId,NameOfPokemon
0,1106.jpg,Bulbasaur
1,2236.jpg,Charmander
2,1131.jpg,Bulbasaur
3,2221.jpg,Charmander
4,2011.jpg,Pikachu
...,...,...
118,1124.jpg,Bulbasaur
119,2014.jpg,Pikachu
120,2235.jpg,Charmander
121,1126.jpg,Bulbasaur


In [0]:
df2.to_csv('prediction2.csv',index =False)